In [1]:
# Standard imports
import os
import json
import sqlite3

# External libraries
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr 

import base64
from io import BytesIO
from PIL import Image

In [2]:
# Load environment variables
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

# OpenAI client setup
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
# SQLite database initialization
DB = "prices.db"
with sqlite3.connect(DB) as conn:
    conn.execute(
        "CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)"
    )
    conn.commit()
    

In [4]:
# Read ticket price from the database
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.execute(
            "SELECT price FROM prices WHERE city = ?", (city.lower(),)
        )
        row = cursor.fetchone()
    return f"Ticket price to {city} is ${row[0]}" if row else "No price data available for this city"

# Write or update ticket price in the database
def set_ticket_price(city, price):
    print(f"DATABASE TOOL CALLED: Setting price for {city} to {price}", flush=True)
    with sqlite3.connect(DB) as conn:
        conn.execute(
            "INSERT INTO prices (city, price) VALUES (?, ?) "
            "ON CONFLICT(city) DO UPDATE SET price = ?",
            (city.lower(), price, price),
        )
        conn.commit()
    return f"Updated price for {city} to ${price}."

# Seed initial data into the DB
ticket_prices = {"london":799, "paris": 899, "tokyo":1400, "sydney":2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

DATABASE TOOL CALLED: Setting price for london to 799
DATABASE TOOL CALLED: Setting price for paris to 899
DATABASE TOOL CALLED: Setting price for tokyo to 1400
DATABASE TOOL CALLED: Setting price for sydney to 2999


In [5]:
# Tool specification: get ticket price
get_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Tool specification: set ticket price
set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a return ticket for a given destination. Call this whenever you need to update the ticket price for a city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city whose ticket price needs to be updated",
            },
            "new_price": {
                "type": "number",
                "description": "The return ticket price to assign to the city",
            },
        },
        "required": ["destination_city", "new_price"],
        "additionalProperties": False,
    },
}

# Combine tool definitions
tools = [
    {"type": "function", "function": get_price_function},
    {"type": "function", "function": set_price_function},
]

In [ ]:
# Execute tool calls made by the model
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get("destination_city")
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "set_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get("destination_city")
            price = arguments.get("new_price")
            confirmation = set_ticket_price(city, price)
            responses.append({
                "role": "tool",
                "content": confirmation,
                "tool_call_id": tool_call.id
            })
    return responses, city

In [ ]:
def artist(city):
    image_response = openai.images.generate(
        model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style"`,
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

def talker(message):
    response = openai.audio.speech.create(
        model="gpt-4o-mini-tts",
        voice="onyx",
        input=message,
    )
    return response.content

In [ ]:
# System prompt for the assistant
system_message = (
    "You are a helpful assistant for an Airline called FlightAI. "
    "Give short, courteous answers, no more than 1 sentence. "
    "Always be accurate. If you don't know the answer, say so."
)

# Chat handler for Gradio and tool-calling loop
def chat(user_message, history):
    # Normalize history from Gradio format to OpenAI format
    history = [{"role": h["role"], "content": h["content"]} for h in history]

    # Start message list with system + history + current user message
    messages = (
        [{"role": "system", "content": system_message}]
        + history
        + [{"role": "user", "content": user_message}]
    )

    # First model call (tools are available)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
    )

    # While the model is asking to call tools
    while response.choices[0].finish_reason == "tool_calls":
        assistant_msg = response.choices[0].message  # assistant message that includes tool_calls

        # Let your Python tools run
        tool_responses, cities = handle_tool_calls(assistant_msg)

        # Add assistant tool-call message and tool outputs to the conversation
        messages.append(assistant_msg)
        messages.extend(tool_responses)

        # Ask the model again, now with tool outputs included
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools,
        )

    # Final assistant message (no more tool calls)
    final_msg = response.choices[0].message.content
    
    voice = talker(voice)
    if cities:
        image = artist(cities[0])
    return final_msg, voice, image 


In [8]:
# Launch simple chat interface
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7902
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Getting price for London
DATABASE TOOL CALLED: Getting price for Tehran
DATABASE TOOL CALLED: Setting price for Tehran to 1199
